设计思路：

假设从某个时间点开始，每周淘汰最后10%。


In [16]:
import pandas as pd
import math
import gc
from IPython.display import display, HTML

import base
import fund_scanner.common_tools.database as db
import fund_scanner.common_tools.myweixin as wx

engine = db.get_sqlalchemy_engine()

In [2]:
# 准备所有参赛者

sql = 'select * from funds a left join funds_update b on a.funds_id=b.funds_id'

df_all_candidates = pd.read_sql(sql, engine).iloc[:,[0,1,2,3,4,5,11]]

df_all_candidates = df_all_candidates.set_index('funds_id')

df_all_candidates['last_price'] = math.nan
df_all_candidates['current_price'] = math.nan
df_all_candidates['gain_ratio'] = math.nan
df_all_candidates['good'] = 0
good = {
    'not_in_yet' : 0,
    'healthy' : 1,
    'hurt_once' : 2,
    'hurt_twice' : 3,
    'out' : 10,
    'noway' : 11
    }
# good值:
#     0 : 未参赛
#     1 : 已参赛，正活跃
#     2 : 受伤 1次
#     3 : 受伤 2次
#     10 : 淘汰
#     11 : 没有参赛资格
# last_price: 上一轮净值
# current_price: 这一轮净值
# gain_ratio: 这一轮涨幅
df_all_candidates['current_ranking'] = 0
df_all_candidates['total_ranking'] = 0

df_all_candidates = df_all_candidates.sort_values('funds_start_date')

df_all_candidates.tail(3)

,funds_code,funds_name_full,funds_type,funds_start_date,update_time,funds_amount,last_price,current_price,gain_ratio,good,current_ranking,total_ranking
funds_id,,,,,,,,,,,,
164451,004993,中欧可转债债券A,债券型,NaT,2017-09-17 12:20:04,NaN,NaN,NaN,NaN,0,0,0
164956,005158,长江乐盈定开债,定开债券,NaT,2017-09-17 20:52:04,NaN,NaN,NaN,NaN,0,0,0
175225,005062,博时中证500指数增强,股票指数,NaT,2017-09-15 15:44:05,NaN,NaN,NaN,NaN,0,0,0


In [3]:
# game1 游戏规则
# 选择 2013年 前成立的老基金们参赛，从 2013年 开始，每 14天 比一次净值的涨幅，
# 把涨幅最靠后 8支 的淘汰，如果剩下基金比较少了，则淘汰排名最后的 1/10
# 一直跑到 today，
# 如果中途淘汰数量过多，最后剩下只有 5支 基金的时候，则直接中止比赛，
# 返回最后留下的基金
# 同时可以观测某一支基金是在哪个过程被淘汰的

def game1(funds_start_date='2013-1-1', competition_start_date='2013-1-1', competition_end_date='today',
          competition_time_span=14, bottom_to_be_out_max_count=8, bottom_to_be_out_percentage=10, 
          min_result_count=5, watching_code=None):
    
    gc.collect()
    # 重新读取数据
    df_competition = df_all_candidates.copy()

    # 2017年1月1日以后成立的基金或者没有写成立时间的基金没有参赛资格
    df_competition.loc[df_competition['funds_start_date']>pd.to_datetime(funds_start_date), 'good'] = good['noway']
    df_competition.loc[df_competition['funds_start_date'].isnull(),'good'] = good['noway']

    #资产规模小于1亿，或者没有数据的没有资格参赛
    df_competition.loc[df_competition['funds_amount']<1, 'good'] = good['noway']
    df_competition.loc[df_competition['funds_amount'].isnull(),'good'] = good['noway']


    #比赛从最早一直基金成立开始
    #start_point = pd.to_datetime(df_competition.iloc[0,3])
    #比赛从2013年1月1日开始
    start_point = pd.to_datetime(competition_start_date)
    round_count = 1
    current_date = start_point
    while current_date < pd.to_datetime(competition_end_date):
        df_current_price = pd.read_sql('select * from funds_historical_price where funds_price_date=\'%s\''%current_date, engine)
        if len(df_current_price)<10:
            current_date = current_date + pd.DateOffset(1)
            continue

        # 把当前价格设为上一期价格
        df_competition['last_price'] = df_competition['current_price']
        # 设置当天价格（如果当天没几个价格就查次日的）
        df_current_price = df_current_price.set_index('funds_id')
        df_competition['current_price'].update(df_current_price['funds_price_adjust'])

        # 如果价格从0到有，则说明参赛了，设置为 参赛1
        df_competition.loc[(df_competition['last_price']>0) & (df_competition['good']==good['not_in_yet']), 'good']=good['healthy']


        # 如果价格从非0到有，则说明价格变化了，计算变化率
        df_competition['gain_ratio'] = \
        ( df_competition['current_price'] - df_competition['last_price'] ) / df_competition['last_price']

        # 按变化率排序，将最后10%选手设置为 淘汰
        active_candidates = len(df_competition.loc[(df_competition['good']==good['healthy'])])
        print('Round %d: %s'% (round_count, current_date))
        if active_candidates>min_result_count:
            should_be_removed = active_candidates // bottom_to_be_out_percentage
            if should_be_removed > bottom_to_be_out_max_count:
                should_be_removed = bottom_to_be_out_max_count
            if should_be_removed>=1:
                current_competition = df_competition.loc[(df_competition['good']==good['healthy'])].sort_values('gain_ratio')
                min_gain_ratio = current_competition['gain_ratio'].min()
                max_gain_ratio = current_competition['gain_ratio'].max()
                threshold = current_competition.iloc[should_be_removed, :]['gain_ratio']
                print('最低涨幅 %f, 最高涨幅 %f ----> 淘汰 %d/%d 名选手，他们当期收益率低于 %f'\
                      %(min_gain_ratio, max_gain_ratio, should_be_removed, active_candidates, threshold))
                df_competition.loc[(df_competition['good']==good['healthy']) & (df_competition['gain_ratio']<=threshold), 'good'] = good['out']
            else:
                break

        round_count += 1
        current_date = current_date + pd.DateOffset(competition_time_span)

        # 观测特定基金的状态
        if watching_code is not None:
            if type(watching_code) == list:
                display(df_competition.loc[df_competition['funds_code'].isin(watching_code), :])
            elif type(watching_code) == str:
                display(df_competition.loc[df_competition['funds_code'] == (watching_code), :])
            else:
                pass

    # The winner is:
    return df_competition.loc[df_competition['good']==good['healthy']].sort_values('current_price', ascending=False)

In [4]:
#Start game1
#game1(competition_start_date='2015-05-23', competition_time_span=5, bottom_to_be_out_max_count=10, watching_code='540006')

In [5]:
# game2 游戏规则
# 选择 2016年 前成立的老基金们参赛，从 2016年 开始，每 14天 比一次净值的涨幅，
# 累加每一次比赛得到的排名
# 一直跑到 today，
# 返回所有参赛基金的排名积分的累加

def game2(funds_start_date='2016-1-1', competition_start_date='2016-1-1', competition_end_date='today',
          competition_time_span=14, watching_code=None):
    
    gc.collect()
    # 重新读取数据
    df_competition = df_all_candidates.copy()

    # 所有队伍先设置状态
    df_competition.loc[:,'good'] = good['healthy']
    
    # 2017年1月1日以后成立的基金或者没有写成立时间的基金没有参赛资格
    df_competition.loc[df_competition['funds_start_date']>pd.to_datetime(funds_start_date), 'good'] = good['noway']
    df_competition.loc[df_competition['funds_start_date'].isnull(),'good'] = good['noway']

    #资产规模小于1亿，或者没有数据的没有资格参赛
    df_competition.loc[df_competition['funds_amount']<1, 'good'] = good['noway']
    df_competition.loc[df_competition['funds_amount'].isnull(),'good'] = good['noway']

    df_competition = df_competition[df_competition['good']==good['healthy']]
    #参赛队伍准备完毕

    
    #比赛从最早一直基金成立开始
    #start_point = pd.to_datetime(df_competition.iloc[0,3])
    #比赛从2013年1月1日开始
    start_point = pd.to_datetime(competition_start_date)
    round_count = 1
    current_date = start_point
    while current_date < pd.to_datetime(competition_end_date):
        df_current_price = pd.read_sql('select * from funds_historical_price where funds_price_date=\'%s\''%current_date, engine)
        if len(df_current_price)<10:
            current_date = current_date + pd.DateOffset(1)
            continue

        # 把当前价格设为上一期价格
        df_competition['last_price'] = df_competition['current_price']
        # 设置当天价格（如果当天没几个价格就查次日的）
        df_current_price = df_current_price.set_index('funds_id')
        df_competition['current_price'].update(df_current_price['funds_price_adjust'])

        # 如果价格从非0到有，则说明价格变化了，计算变化率
        df_competition['gain_ratio'] = \
        ( df_competition['current_price'] - df_competition['last_price'] ) / df_competition['last_price']

        # 按变化率排序，累加排名
        df_competition = df_competition.sort_values('gain_ratio', ascending=True)
        df_competition = df_competition.assign(current_ranking=[i for i in range(len(df_competition))])
        df_competition.loc[df_competition['gain_ratio']>-100, 'total_ranking'] = df_competition['total_ranking'] + df_competition['current_ranking'] - len(df_competition)/2
        
        print('Round %d: %s'% (round_count, current_date))

        round_count += 1
        current_date = current_date + pd.DateOffset(competition_time_span)

        # 观测特定基金的状态
        if watching_code is not None:
            if type(watching_code) == list:
                display(df_competition.loc[df_competition['funds_code'].isin(watching_code), :].sort_index())
            elif type(watching_code) == str:
                display(df_competition.loc[df_competition['funds_code'] == (watching_code), :])
            else:
                pass


    # The winner is (return the full list, and the winner is on the top with highest ranking):
    return df_competition.loc[df_competition['good']==good['healthy']].sort_values('total_ranking', ascending=False)

df = game2()
df.head(20)

Round 1: 2016-01-04 00:00:00
Round 2: 2016-01-18 00:00:00
Round 3: 2016-02-01 00:00:00
Round 4: 2016-02-15 00:00:00
Round 5: 2016-02-29 00:00:00
Round 6: 2016-03-14 00:00:00
Round 7: 2016-03-28 00:00:00
Round 8: 2016-04-11 00:00:00
Round 9: 2016-04-25 00:00:00
Round 10: 2016-05-09 00:00:00
Round 11: 2016-05-23 00:00:00
Round 12: 2016-06-06 00:00:00
Round 13: 2016-06-20 00:00:00
Round 14: 2016-07-04 00:00:00
Round 15: 2016-07-18 00:00:00
Round 16: 2016-08-01 00:00:00
Round 17: 2016-08-15 00:00:00
Round 18: 2016-08-29 00:00:00
Round 19: 2016-09-12 00:00:00
Round 20: 2016-09-26 00:00:00
Round 21: 2016-10-10 00:00:00
Round 22: 2016-10-24 00:00:00
Round 23: 2016-11-07 00:00:00
Round 24: 2016-11-21 00:00:00
Round 25: 2016-12-05 00:00:00
Round 26: 2016-12-19 00:00:00
Round 27: 2017-01-03 00:00:00
Round 28: 2017-01-17 00:00:00
Round 29: 2017-02-03 00:00:00
Round 30: 2017-02-17 00:00:00
Round 31: 2017-03-03 00:00:00
Round 32: 2017-03-17 00:00:00
Round 33: 2017-03-31 00:00:00
Round 34: 2017-04-1

,funds_code,funds_name_full,funds_type,funds_start_date,update_time,funds_amount,last_price,current_price,gain_ratio,good,current_ranking,total_ranking
funds_id,,,,,,,,,,,,
121624,150230,鹏华酒分级B,分级杠杆,2015-04-29,2017-09-16 05:10:03,2.92,1.2800,1.2960,0.012500,1,2159,3101.5
121925,502015,长盛中证申万一带一路分级B,分级杠杆,2015-05-29,2017-09-16 10:14:03,17.56,0.4165,0.4265,0.024010,1,2290,2407.5
5185,000970,东方红睿元混合,混合型,2015-01-21,2017-09-16 20:45:03,13.86,2.0190,2.0190,0.000000,1,734,2047.0
3914,000577,安信价值精选股票,股票型,2014-04-21,2017-09-15 01:34:03,27.34,2.6450,2.6750,0.011342,1,2135,2039.0
121623,150238,鹏华环保分级B,分级杠杆,2015-06-16,2017-09-16 05:09:04,1.46,0.4100,0.4390,0.070732,1,2337,2015.5
121772,150056,工银中证500分级B,分级杠杆,2012-01-31,2017-09-16 07:40:04,1.04,1.5332,1.5642,0.020219,1,2264,1998.5
121709,159944,广发中证全指原材料ETF,ETF-场内,2015-06-25,2017-09-16 06:37:04,1.50,1.0152,1.0234,0.008077,1,2035,1909.5
121594,510130,易方达上证中盘ETF,ETF-场内,2010-03-29,2017-09-16 04:40:03,2.61,1.3842,1.3874,0.002312,1,1612,1896.5
121388,150191,新华中证环保产业指数分级B,分级杠杆,2014-09-11,2017-09-16 01:11:03,2.37,0.7850,0.8380,0.067516,1,2336,1887.5


In [17]:
winner = df.head(10).loc[:, ['funds_code', 'funds_name_full']]
msg = ''
for i,r in winner.iterrows():
    msg += '\n' + (r['funds_code'] + ' => ' + r['funds_name_full'])
    
print(msg)

wx.send(msg)


150230 => 鹏华酒分级B
502015 => 长盛中证申万一带一路分级B
000970 => 东方红睿元混合
000577 => 安信价值精选股票
150238 => 鹏华环保分级B
150056 => 工银中证500分级B
159944 => 广发中证全指原材料ETF
510130 => 易方达上证中盘ETF
150191 => 新华中证环保产业指数分级B
159935 => 景顺长城中证500ETF
